# Imports

In [1]:
%autosave 60

Autosaving every 60 seconds


In [2]:
#import spacy
import numpy
import seaborn
import matplotlib.pyplot as plt
import pandas
import pickle
from pathlib import Path
from dataclasses import dataclass, field
from tqdm import tqdm
import transformers # conda install tensorflow
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
from IPython.display import display, HTML
from collections import deque
from multiprocessing import Pool
import os
from functools import partial
import pandas as pd
import numpy as np
import re

# Needed to train the model
from datasets import Dataset,DatasetDict
from transformers import AutoModelForSequenceClassification,AutoTokenizer

/Users/stefano/opt/anaconda3/envs/s2ds/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data

Read

In [3]:
inputdir = Path('../dataset_for_inference')
filename = Path('portuguese_data_for_inference.csv')
data = pd.read_csv(inputdir/filename)

Take a look at it

In [4]:
data

,id,name,full_text,article_length
0,54006424,Clemens Tönnies,O presidente do FC Schalke 04 e proprietário ...,6376
1,54006424,Gerald Asamoah,O presidente do FC Schalke 04 e proprietário ...,6376
2,54006424,Hans Sarpei,O presidente do FC Schalke 04 e proprietário ...,6376
3,54006424,Cacau,O presidente do FC Schalke 04 e proprietário ...,6376
4,54003004,Angela Merkel,"Em meados de março, quando o coronavírus com...",5029
...,...,...,...,...
40136,54005845,Von der Leyen,A controversa lei de segurança nacional impos...,5983
40137,54005845,Charles Michel,A controversa lei de segurança nacional impos...,5983
40138,54005845,Gyde Jensen,A controversa lei de segurança nacional impos...,5983
40139,54005845,Angela Merkel,A controversa lei de segurança nacional impos...,5983


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40141 entries, 0 to 40140
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              40141 non-null  int64 
 1   name            40141 non-null  object
 2   full_text       40141 non-null  object
 3   article_length  40141 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 1.2+ MB


In [6]:
data.describe()

,id,article_length
count,4.014100e+04,40141.000000
mean,5.723879e+07,4690.522209
std,3.093354e+06,1853.511628
min,3.672863e+07,816.000000
25%,5.464829e+07,3287.000000
50%,5.712484e+07,4438.000000
75%,5.994633e+07,5818.000000
max,6.246369e+07,11293.000000


In [84]:
data.describe(include=['object'])

,name,full_text
count,40141,40141
unique,16669,9332
top,Jair Bolsonaro,Os Estados Unidos impuseram nesta sexta-feira ...
freq,1530,10


# Remove unnecessary columns

In [85]:
data = data.drop(columns=['article_length'])

In [86]:
data

,id,name,full_text
0,54006424,Clemens Tönnies,O presidente do FC Schalke 04 e proprietário ...
1,54006424,Gerald Asamoah,O presidente do FC Schalke 04 e proprietário ...
2,54006424,Hans Sarpei,O presidente do FC Schalke 04 e proprietário ...
3,54006424,Cacau,O presidente do FC Schalke 04 e proprietário ...
4,54003004,Angela Merkel,"Em meados de março, quando o coronavírus com..."
...,...,...,...
40136,54005845,Von der Leyen,A controversa lei de segurança nacional impos...
40137,54005845,Charles Michel,A controversa lei de segurança nacional impos...
40138,54005845,Gyde Jensen,A controversa lei de segurança nacional impos...
40139,54005845,Angela Merkel,A controversa lei de segurança nacional impos...


# Check that all names are actually in the article text

In [87]:
names_not_in_text=[]
for row in data.itertuples():
    if not row.name in row.full_text:
        names_not_in_text.append(row.Index)

In [88]:
data.loc[names_not_in_text]

,id,name,full_text
70,54029587,Annegret Kramp - Karrenbauer,"A ministra alemã da Defesa, Annegret Kramp-Ka..."
106,54026231,Yedi López - Cotarelo,Cuba retorna lentamente ao normal. Em todo o p...
129,54036868,Jean - Yves Le Drian,"O primeiro-ministro francês, Edouard Philippe..."
152,54037037,M. O,Decisões do Supremo Tribunal Federal (STF) in...
166,54044041,Jean - Yves Drian,Às 10 horas da manhã desta sexta-feira (03/0...
...,...,...,...
39921,53932221,Claudia Azevedo - Ramos,A Amazônia tem 23% de suas terras públicas n...
39925,53932071,Kim Yo - jong,"Em 24 de junho de 1950, tanques norte-coreanos..."
39926,53932071,Kim Jong - un,"Em 24 de junho de 1950, tanques norte-coreanos..."
40061,53979535,Jean - Luc Mélenchon,O partido do presidente Emmanuel Macron sofreu...


It seems that there is a difficulty when dashes are part of the name. **This has to do with the NER, which may have returned a different unicode (compared to the one in the original article text) when extarcting the names**.

Let us see if that is the whole issue.

In [89]:
names_with_dash=[]
for row in data.itertuples():
    if ' - ' in row.name:
        names_with_dash.append(row.Index)

In [90]:
len(names_with_dash)

1088

In [91]:
data.loc[list(set(names_not_in_text)-set(names_with_dash))]

,id,name,full_text
22656,61969974,"Federico "" Fico "" Gutiérrez",O candidato de esquerda Gustavo Petro e o empr...
39044,53704930,Robert O ’ Brien,"O presidente dos Estados Unidos, Donald Trump,..."
23300,62111436,""" Pelado """,A Polícia Federal (PF) afirmou neste domingo ...
5508,55466164,Jaroslav Nad ',O governo da Eslováquia informou neste doming...
25991,58710325,Fernando Henrique [ Cardoso,"Para o cineasta Luiz Bolognesi, o fogo que ati..."
...,...,...,...
26618,58866305,"Jorge "" Tigre "" Acosta",A justiça argentina sentenciou nesta sexta-fe...
26619,58866305,"Alberto "" Gato "" González",A justiça argentina sentenciou nesta sexta-fe...
18941,61085218,Fernanda *,"""Eu já vi mulher defecar no chão durante a r..."
3070,54838911,Abraham ] Weintraub,"Pouco mais de dois anos atrás, quando o diret..."


It seems that are 61 names without dashes in it that are also problematic. Judging by the table above, those seems to be cases where the NER got confused. 

Fortunatelly, all cases in which the name is "not in the article text" amount to 1149 cases. Thas corresponds to a small percentage of the inference data.

In [7]:
print(f'{100*(1149/40141)}%')

2.862410004733315%


We will remove those 1149 cases

In [93]:
len(names_not_in_text)

1149

## Remove problematic inputs

In [94]:
data = data.drop(names_not_in_text)

Check result

In [95]:
names_not_in_text=[]
for row in data.itertuples():
    if not row.name in row.full_text:
        names_not_in_text.append(row.Index)

In [96]:
len(names_not_in_text)

0

In [97]:
data = data.reset_index(drop=True)

In [98]:
data

,id,name,full_text
0,54006424,Clemens Tönnies,O presidente do FC Schalke 04 e proprietário ...
1,54006424,Gerald Asamoah,O presidente do FC Schalke 04 e proprietário ...
2,54006424,Hans Sarpei,O presidente do FC Schalke 04 e proprietário ...
3,54006424,Cacau,O presidente do FC Schalke 04 e proprietário ...
4,54003004,Angela Merkel,"Em meados de março, quando o coronavírus com..."
...,...,...,...
38987,54005845,Von der Leyen,A controversa lei de segurança nacional impos...
38988,54005845,Charles Michel,A controversa lei de segurança nacional impos...
38989,54005845,Gyde Jensen,A controversa lei de segurança nacional impos...
38990,54005845,Angela Merkel,A controversa lei de segurança nacional impos...


# Conform to use input string whose length does not exceed 512 characters

In [99]:
new_full_text = []
for row in data.itertuples():
    name = row.name
    full_text = row.full_text
    
    # Split full text into sentences
    sentences = full_text.split('. ')
    for i in range(len(sentences)-1):
        sentences[i] = sentences[i]+'.' # Add dot that was removed when splitting

    # Find name occurrences
    name_occurrences = []
    for i in range(len(sentences)):
        if name in sentences[i]:
            name_occurrences.append(i)
    
    # First ocurrence
    name_occurrence = np.min(name_occurrences)
    
    # Append sentences one by one until it gets too large
    max_num_of_characters = 420
    max_num_of_sentences_to_append = 7
    new_sentence = sentences[name_occurrence]
    for i in range(2,max_num_of_sentences_to_append+2):
        _new_sentence = ' '.join(sentences[name_occurrence:name_occurrence+i])
        if len(_new_sentence) <= max_num_of_characters:
            new_sentence = _new_sentence
            
    # Collect new input
    new_full_text.append(new_sentence)

In [100]:
data['new_full_text'] = new_full_text

In [101]:
data

,id,name,full_text,new_full_text
0,54006424,Clemens Tönnies,O presidente do FC Schalke 04 e proprietário ...,O presidente do FC Schalke 04 e proprietário ...
1,54006424,Gerald Asamoah,O presidente do FC Schalke 04 e proprietário ...,"No mês de agosto, durante um evento do clube,..."
2,54006424,Hans Sarpei,O presidente do FC Schalke 04 e proprietário ...,"No mês de agosto, durante um evento do clube,..."
3,54006424,Cacau,O presidente do FC Schalke 04 e proprietário ...,"No mês de agosto, durante um evento do clube,..."
4,54003004,Angela Merkel,"Em meados de março, quando o coronavírus com...","Em meados de março, quando o coronavírus com..."
...,...,...,...,...
38987,54005845,Von der Leyen,A controversa lei de segurança nacional impos...,Von der Leyen acrescentou que a UE discute med...
38988,54005845,Charles Michel,A controversa lei de segurança nacional impos...,"O presidente do Conselho Europeu, Charles Mich..."
38989,54005845,Gyde Jensen,A controversa lei de segurança nacional impos...,"Na Alemanha, a presidente do Comitê de Direit..."
38990,54005845,Angela Merkel,A controversa lei de segurança nacional impos...,"Na Alemanha, a presidente do Comitê de Direit..."


Check

In [102]:
i=150
print(data.iloc[i]['name'])
print('')
print(data.iloc[i].full_text)
print('')
print(data.iloc[i].new_full_text)
print('')
print(len(data.iloc[i].new_full_text))

Priscila Naves Tardelli

Decisões do Supremo Tribunal Federal (STF) influenciam todas as instâncias inferiores do Poder Judiciário. Por esse motivo, causou repercussão entre advogados criminalistas o fato de, especialmente no contexto da atual pandemia, a ministra do STF Rosa Weber negar, na última terça-feira (30/06), um pedido de liminar para que um jovem condenado pelo furto de dois xampus, no valor de 10 reais cada, cumprisse penas alternativas. O jovem em questão tem outros furtos na ficha de antecedentes. Em sua decisão, a ministra destaca trechos da condenação pelo juiz de primeira instância, que se baseou na reincidência para afirmar que o jovem não tem condições de "conviver em sociedade", ainda que o valor dos xampus seja irrisório. A decisão intensifica um debate que tem criado divisões no meio jurídico durante a pandemia. No dia 17 de março, quando os números da covid-19 no país ainda estavam longe dos atuais, o Conselho Nacional de Justiça (CNJ) editou a Recomendação 62/20

Drop and rename columns

In [105]:
data = data.drop(columns=['full_text'])
data = data.rename(columns={"new_full_text": "full_text"})

In [106]:
data.head()

,id,name,full_text
0,54006424,Clemens Tönnies,O presidente do FC Schalke 04 e proprietário ...
1,54006424,Gerald Asamoah,"No mês de agosto, durante um evento do clube,..."
2,54006424,Hans Sarpei,"No mês de agosto, durante um evento do clube,..."
3,54006424,Cacau,"No mês de agosto, durante um evento do clube,..."
4,54003004,Angela Merkel,"Em meados de março, quando o coronavírus com..."


Create the input

In [107]:
data['input'] = 'TEXT1: ' + data['name'] + '; TEXT2: ' + data['full_text']

In [108]:
data.head()

,id,name,full_text,input
0,54006424,Clemens Tönnies,O presidente do FC Schalke 04 e proprietário ...,TEXT1: Clemens Tönnies; TEXT2: O presidente d...
1,54006424,Gerald Asamoah,"No mês de agosto, durante um evento do clube,...",TEXT1: Gerald Asamoah; TEXT2: No mês de agost...
2,54006424,Hans Sarpei,"No mês de agosto, durante um evento do clube,...","TEXT1: Hans Sarpei; TEXT2: No mês de agosto, ..."
3,54006424,Cacau,"No mês de agosto, durante um evento do clube,...","TEXT1: Cacau; TEXT2: No mês de agosto, durant..."
4,54003004,Angela Merkel,"Em meados de março, quando o coronavírus com...",TEXT1: Angela Merkel; TEXT2: Em meados de marc...


Check

In [109]:
i=100
print(data.iloc[i]['name'])
print('')
print(data.iloc[i].full_text)
print('')
print(len(data.iloc[i].full_text))
print('')
print(data.iloc[i].input)
print('')
print(len(data.iloc[i].input))

patriarca Bartolomeu de

O patriarca Bartolomeu de Constantinopla, chefe da Igreja Ortodoxa, se disse estritamente contra a conversão de Hagia Sophia em uma mesquita. "A Hagia é um dos monumentos mais importantes da civilização humana e pertence não apenas aos seus proprietários imediatos, mas a toda a humanidade", disse, durante uma missa em Istambul.

335

TEXT1: patriarca Bartolomeu de; TEXT2: O patriarca Bartolomeu de Constantinopla, chefe da Igreja Ortodoxa, se disse estritamente contra a conversão de Hagia Sophia em uma mesquita. "A Hagia é um dos monumentos mais importantes da civilização humana e pertence não apenas aos seus proprietários imediatos, mas a toda a humanidade", disse, durante uma missa em Istambul.

374


# Format the dataset

Keep only columns needed for the model 

In [33]:
data = data[['input']]

In [34]:
data.head()

,input
0,TEXT1: Clemens Tönnies; TEXT2: O presidente d...
1,TEXT1: Gerald Asamoah; TEXT2: No mês de agost...
2,"TEXT1: Hans Sarpei; TEXT2: No mês de agosto, ..."
3,"TEXT1: Cacau; TEXT2: No mês de agosto, durant..."
4,TEXT1: Angela Merkel; TEXT2: Em meados de marc...


# Read model

**I still do not know how to properly load a saved model, so I will circunvent the issue by repeating the same steps I used for training the model but skipping the actual training part**

## Read dataset

In [8]:
inputdir = Path('../training')

filename = Path('portuguese_training_data_v5.csv')
training_data = pd.read_csv(inputdir/filename)

filename = Path('portuguese_test_data_v5.csv')
test_data = pd.read_csv(inputdir/filename)

In [9]:
training_data

,id,name,full_text,label,input
0,54153414,Andrzej Duda,A comissão eleitoral da Polônia anunciou nes...,1.0,TEXT1: Andrzej Duda; TEXT2: A comissão eleito...
1,55253572,Erich Honecker,"Natural e casual, elegantemente trajando ócul...",1.0,TEXT1: Erich Honecker; TEXT2: Natural e casual...
2,58168985,Chris Whitty,Apesar do número crescente de casos de covid-...,0.0,TEXT1: Chris Whitty; TEXT2: Apesar do número ...
3,53821497,Luiz Henrique Mandetta,"A Food and Drug Administration (FDA), agência...",1.0,TEXT1: Luiz Henrique Mandetta; TEXT2: A Food a...
4,57170897,Angela Merkel,O populismo de direita alemão deu um passo ma...,0.0,TEXT1: Angela Merkel; TEXT2: O populismo de di...
...,...,...,...,...,...
7835,61292150,Dilma Rousseff,Trata-se de um movimento iniciado nos anos 199...,0.0,TEXT1: Dilma Rousseff; TEXT2: Trata-se de um m...
7836,60338691,Thomas Mann,"A bordo de jet-skis, os homens avançam velozm...",1.0,TEXT1: Thomas Mann; TEXT2: A bordo de jet-skis...
7837,57352798,Donald Trump,Os desenvolvedores da vacina russa contra covi...,1.0,TEXT1: Donald Trump; TEXT2: Os desenvolvedores...
7838,61662604,Bolsonaro,Durante o governo do ex-presidente americano D...,1.0,TEXT1: Bolsonaro; TEXT2: Durante o governo do ...


## Conform to use input string whose length does not exceed 512 characters

In [37]:
new_full_text = []
for row in training_data.itertuples():
    name = row.name
    full_text = row.full_text
    
    # Split full text into sentences
    sentences = full_text.split('. ')
    for i in range(len(sentences)-1):
        sentences[i] = sentences[i]+'.' # Add dot that was removed when splitting

    # Find name occurrences
    name_occurrences = []
    for i in range(len(sentences)):
        if name in sentences[i]:
            name_occurrences.append(i)
    
    # First ocurrence
    name_occurrence = np.min(name_occurrences)
    
    # Append sentences one by one until it gets too large
    max_num_of_characters = 420
    max_num_of_sentences_to_append = 7
    new_sentence = sentences[name_occurrence]
    for i in range(2,max_num_of_sentences_to_append+2):
        _new_sentence = ' '.join(sentences[name_occurrence:name_occurrence+i])
        if len(_new_sentence) <= max_num_of_characters:
            new_sentence = _new_sentence
            
    # Collect new input
    new_full_text.append(new_sentence)

In [38]:
training_data['new_full_text'] = new_full_text

In [39]:
training_data.head()

,id,name,full_text,label,input,new_full_text
0,54153414,Andrzej Duda,A comissão eleitoral da Polônia anunciou nes...,1.0,TEXT1: Andrzej Duda; TEXT2: A comissão eleito...,A comissão eleitoral da Polônia anunciou nes...
1,55253572,Erich Honecker,"Natural e casual, elegantemente trajando ócul...",1.0,TEXT1: Erich Honecker; TEXT2: Natural e casual...,"Natural e casual, elegantemente trajando ócul..."
2,58168985,Chris Whitty,Apesar do número crescente de casos de covid-...,0.0,TEXT1: Chris Whitty; TEXT2: Apesar do número ...,Questionados sobre isso durante a coletiva de ...
3,53821497,Luiz Henrique Mandetta,"A Food and Drug Administration (FDA), agência...",1.0,TEXT1: Luiz Henrique Mandetta; TEXT2: A Food a...,Discórdias sobre o uso do medicamento teriam ...
4,57170897,Angela Merkel,O populismo de direita alemão deu um passo ma...,0.0,TEXT1: Angela Merkel; TEXT2: O populismo de di...,"Em setembro, a Alemanha realiza eleições ger..."


Check

In [40]:
i=150
print(training_data.iloc[i]['name'])
print('')
print(training_data.iloc[i].full_text)
print('')
print(training_data.iloc[i].new_full_text)
print('')
print(len(training_data.iloc[i].new_full_text))

Heiko Maas

Em videoconferência no Conselho de Segurança das Nações Unidas, o ministro alemão do Exterior, Heiko Maas, conclamou a comunidade mundial a agir de forma mais decidida contra todas as formas de violência sexual no contexto de conflitos de guerra. "Estupro, prostituição forçada e escravidão sexual continuam sendo empregados como armas em conflitos por todo o mundo", frisou o chefe de diplomacia. Ele criticou que, 20 anos após a aprovação da Resolução 1.325 da ONU e mais de um ano após a Resolução 2.467, "quase não se veem progressos": as vítimas frequentemente não recebem a ajuda de que necessitam, e a criminalidade sexualizada nas guerras segue destruindo sociedades em todas as regiões. Detentora da presidência rotativa do Conselho de Segurança da ONU no mês de julho, a Alemanha declarou como uma de suas prioridades o combate à violência sexual em conflitos bélicos. O debate desta sexta-feira (17/07) foi promovido também pela República Dominicana. Maas reivindicou a impleme

Drop and rename columns

In [43]:
training_data = training_data.drop(columns=['input', 'full_text'])
training_data = training_data.rename(columns={"new_full_text": "full_text"})

In [44]:
training_data.head()

,id,name,label,full_text
0,54153414,Andrzej Duda,1.0,A comissão eleitoral da Polônia anunciou nes...
1,55253572,Erich Honecker,1.0,"Natural e casual, elegantemente trajando ócul..."
2,58168985,Chris Whitty,0.0,Questionados sobre isso durante a coletiva de ...
3,53821497,Luiz Henrique Mandetta,1.0,Discórdias sobre o uso do medicamento teriam ...
4,57170897,Angela Merkel,0.0,"Em setembro, a Alemanha realiza eleições ger..."


Create, once again, the input

In [45]:
training_data['input'] = 'TEXT1: ' + training_data['name'] + '; TEXT2: ' + training_data['full_text']

In [46]:
training_data.head()

,id,name,label,full_text,input
0,54153414,Andrzej Duda,1.0,A comissão eleitoral da Polônia anunciou nes...,TEXT1: Andrzej Duda; TEXT2: A comissão eleito...
1,55253572,Erich Honecker,1.0,"Natural e casual, elegantemente trajando ócul...",TEXT1: Erich Honecker; TEXT2: Natural e casual...
2,58168985,Chris Whitty,0.0,Questionados sobre isso durante a coletiva de ...,TEXT1: Chris Whitty; TEXT2: Questionados sobre...
3,53821497,Luiz Henrique Mandetta,1.0,Discórdias sobre o uso do medicamento teriam ...,TEXT1: Luiz Henrique Mandetta; TEXT2: Discórd...
4,57170897,Angela Merkel,0.0,"Em setembro, a Alemanha realiza eleições ger...","TEXT1: Angela Merkel; TEXT2: Em setembro, a Al..."


Check

In [47]:
i=100
print(training_data.iloc[i]['name'])
print('')
print(training_data.iloc[i].full_text)
print('')
print(len(training_data.iloc[i].full_text))
print('')
print(training_data.iloc[i].input)
print('')
print(len(training_data.iloc[i].input))

Winfried Kretschmann

Segundo o governador Winfried Kretschmann, o tráfego de fronteira com áreas de risco do coronavírus será severamente controlado. Quem trabalha nos países vizinhos poderá atravessar a fronteira para esse fim, mas ficam vedadas as compras internacionais. O político advertiu que muitos deixam de obedecer os regulamentos, e que para desacelerar o avanço dos contágios com o coronavírus, cada dia conta.

410

TEXT1: Winfried Kretschmann; TEXT2: Segundo o governador Winfried Kretschmann, o tráfego de fronteira com áreas de risco do coronavírus será severamente controlado. Quem trabalha nos países vizinhos poderá atravessar a fronteira para esse fim, mas ficam vedadas as compras internacionais. O político advertiu que muitos deixam de obedecer os regulamentos, e que para desacelerar o avanço dos contágios com o coronavírus, cada dia conta.

446


## Format the dataset

Conform to Transformers standard dataset format.

In [48]:
training_ds = Dataset.from_pandas(training_data)

Take a look at the dataset

In [49]:
training_ds

Dataset({
    features: ['id', 'name', 'label', 'full_text', 'input'],
    num_rows: 7840
})

## Tokenization

In [50]:
training_data

,id,name,label,full_text,input
0,54153414,Andrzej Duda,1.0,A comissão eleitoral da Polônia anunciou nes...,TEXT1: Andrzej Duda; TEXT2: A comissão eleito...
1,55253572,Erich Honecker,1.0,"Natural e casual, elegantemente trajando ócul...",TEXT1: Erich Honecker; TEXT2: Natural e casual...
2,58168985,Chris Whitty,0.0,Questionados sobre isso durante a coletiva de ...,TEXT1: Chris Whitty; TEXT2: Questionados sobre...
3,53821497,Luiz Henrique Mandetta,1.0,Discórdias sobre o uso do medicamento teriam ...,TEXT1: Luiz Henrique Mandetta; TEXT2: Discórd...
4,57170897,Angela Merkel,0.0,"Em setembro, a Alemanha realiza eleições ger...","TEXT1: Angela Merkel; TEXT2: Em setembro, a Al..."
...,...,...,...,...,...
7835,61292150,Dilma Rousseff,0.0,O pesquisador situa o governo Fernando Henriqu...,TEXT1: Dilma Rousseff; TEXT2: O pesquisador si...
7836,60338691,Thomas Mann,1.0,"Além disso, ela mesma traduzia para o portugu...","TEXT1: Thomas Mann; TEXT2: Além disso, ela me..."
7837,57352798,Donald Trump,1.0,A mesma sequência de tuítes na conta oficial...,TEXT1: Donald Trump; TEXT2: A mesma sequência...
7838,61662604,Bolsonaro,1.0,A postura oficial do Brasil tem sido favoráve...,TEXT1: Bolsonaro; TEXT2: A postura oficial do ...


Point to our model

In [51]:
model_nm = '/Users/stefano/bert-base-multilingual-cased-ner-hrl-namesGenderPTv2'
tokz = AutoTokenizer.from_pretrained(model_nm)

We can check the indices of the special token IDs like so

In [52]:
tokz.all_special_tokens

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']

Function to tokenize the input

In [53]:
def tok_func(x):
    return tokz (x["input"])

Tokenize the dataset and remove columns that are no longer needed

In [54]:
training_ds

Dataset({
    features: ['id', 'name', 'label', 'full_text', 'input'],
    num_rows: 7840
})

In [55]:
cols = ['id', 'name', 'full_text', 'input']
tok_training_ds = training_ds.map(tok_func, batched=True, remove_columns=cols)

Parameter 'function'=<function tok_func at 0x7fc7d951b130> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
 88%|████████▊ | 7/8 [00:00<00:00,  8.27ba/s]


Take a look at the tokenized dataset

In [56]:
tok_training_ds

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 7840
})

## Separate the training set and the validation set

Use a 25% of the dataset as test set (**same values and seed we used before**, in the Jupyter notebook `training/portuguese_training_v3.ipynb`)

In [57]:
dds = tok_training_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5880
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1960
    })
})

## Training of the model

(**Note that we are not actually gonna train the model**)

Transformers expects metrics to be returned as a `dict`, since that way the trainer knows what label to use, so let's create a function to do that:

In [58]:
from transformers import TrainingArguments, Trainer

In [59]:
epochs = 4
lr = 1e-4 # Learning rate
bs = 128 # batch size
wd = 0.01 # weight decay

Prepare the argument. I am using the same arguments as in the example in the video (1:15:50): https://www.youtube.com/watch?v=toUgBQv1BT8&t=652s

In [60]:
# It gave me an error when `fp16=True` so I use `fp16=False`

args = TrainingArguments('outputs',\
                         learning_rate=lr,\
                         warmup_ratio=0.1,\
                         lr_scheduler_type='cosine',\
                         fp16=False,\
                         evaluation_strategy="epoch",\
                         per_device_train_batch_size=bs,\
                         per_device_eval_batch_size=bs*2,\
                         num_train_epochs=epochs,\
                         weight_decay=wd,\
                         report_to='none')

In [61]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1, ignore_mismatched_sizes=True)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'], tokenizer=tokz)

# Inference

Format the data to be used

In [62]:
eval_ds = Dataset.from_pandas(data).map(tok_func, batched=True)

 97%|█████████▋| 38/39 [00:03<00:00, 12.22ba/s]


In [63]:
eval_ds

Dataset({
    features: ['input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 38992
})

Use the previously trained model to infer/predict the values in the data set

In [64]:
preds = trainer.predict(eval_ds).predictions.astype(float)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: input. If input are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 38992
  Batch size = 256
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Clip and round predictions so that they are exactly 0 or 1

In [68]:
preds_lst = []
for n in np.clip(preds, 0, 1):
    #print(f'{n.item():.2f}')
    preds_lst.append(np.rint(n.item()))

Show it

In [69]:
preds_lst[0:10]

[1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0]

Add the predictions to a DataFrame

In [71]:
data_with_prediction = data.copy()
data_with_prediction['prediction'] = preds_lst

In [73]:
data_with_prediction

,input,prediction
0,TEXT1: Clemens Tönnies; TEXT2: O presidente d...,1.0
1,TEXT1: Gerald Asamoah; TEXT2: No mês de agost...,1.0
2,"TEXT1: Hans Sarpei; TEXT2: No mês de agosto, ...",1.0
3,"TEXT1: Cacau; TEXT2: No mês de agosto, durant...",1.0
4,TEXT1: Angela Merkel; TEXT2: Em meados de marc...,0.0
...,...,...
38987,TEXT1: Von der Leyen; TEXT2: Von der Leyen acr...,1.0
38988,TEXT1: Charles Michel; TEXT2: O presidente do ...,1.0
38989,"TEXT1: Gyde Jensen; TEXT2: Na Alemanha, a pres...",0.0
38990,"TEXT1: Angela Merkel; TEXT2: Na Alemanha, a pr...",0.0


Check the predictions

In [79]:
data_with_prediction.prediction.value_counts()

1.0    29885
0.0     9107
Name: prediction, dtype: int64

# Save inferences

In [76]:
outputdir = Path.cwd()
filename = 'portuguese_all_processed_articles.csv'
data_with_prediction.to_csv(outputdir/filename)

# Add useful columns to the result

Check the two DataFrames to combine

In [110]:
data

,id,name,full_text,input
0,54006424,Clemens Tönnies,O presidente do FC Schalke 04 e proprietário ...,TEXT1: Clemens Tönnies; TEXT2: O presidente d...
1,54006424,Gerald Asamoah,"No mês de agosto, durante um evento do clube,...",TEXT1: Gerald Asamoah; TEXT2: No mês de agost...
2,54006424,Hans Sarpei,"No mês de agosto, durante um evento do clube,...","TEXT1: Hans Sarpei; TEXT2: No mês de agosto, ..."
3,54006424,Cacau,"No mês de agosto, durante um evento do clube,...","TEXT1: Cacau; TEXT2: No mês de agosto, durant..."
4,54003004,Angela Merkel,"Em meados de março, quando o coronavírus com...",TEXT1: Angela Merkel; TEXT2: Em meados de marc...
...,...,...,...,...
38987,54005845,Von der Leyen,Von der Leyen acrescentou que a UE discute med...,TEXT1: Von der Leyen; TEXT2: Von der Leyen acr...
38988,54005845,Charles Michel,"O presidente do Conselho Europeu, Charles Mich...",TEXT1: Charles Michel; TEXT2: O presidente do ...
38989,54005845,Gyde Jensen,"Na Alemanha, a presidente do Comitê de Direit...","TEXT1: Gyde Jensen; TEXT2: Na Alemanha, a pres..."
38990,54005845,Angela Merkel,"Na Alemanha, a presidente do Comitê de Direit...","TEXT1: Angela Merkel; TEXT2: Na Alemanha, a pr..."


In [111]:
data_with_prediction

,input,prediction
0,TEXT1: Clemens Tönnies; TEXT2: O presidente d...,1.0
1,TEXT1: Gerald Asamoah; TEXT2: No mês de agost...,1.0
2,"TEXT1: Hans Sarpei; TEXT2: No mês de agosto, ...",1.0
3,"TEXT1: Cacau; TEXT2: No mês de agosto, durant...",1.0
4,TEXT1: Angela Merkel; TEXT2: Em meados de marc...,0.0
...,...,...
38987,TEXT1: Von der Leyen; TEXT2: Von der Leyen acr...,1.0
38988,TEXT1: Charles Michel; TEXT2: O presidente do ...,1.0
38989,"TEXT1: Gyde Jensen; TEXT2: Na Alemanha, a pres...",0.0
38990,"TEXT1: Angela Merkel; TEXT2: Na Alemanha, a pr...",0.0


Confirm that we are combining the right DataFrames

In [113]:
np.all(data_with_prediction.index == data.index)

True

Drop unnecessary columns

In [115]:
data = data.drop(columns=['full_text'])

Add column for the predictions/inferences

In [116]:
data['prediction'] = preds_lst

Check the two DataFrames to combine

In [117]:
data_with_prediction

,input,prediction
0,TEXT1: Clemens Tönnies; TEXT2: O presidente d...,1.0
1,TEXT1: Gerald Asamoah; TEXT2: No mês de agost...,1.0
2,"TEXT1: Hans Sarpei; TEXT2: No mês de agosto, ...",1.0
3,"TEXT1: Cacau; TEXT2: No mês de agosto, durant...",1.0
4,TEXT1: Angela Merkel; TEXT2: Em meados de marc...,0.0
...,...,...
38987,TEXT1: Von der Leyen; TEXT2: Von der Leyen acr...,1.0
38988,TEXT1: Charles Michel; TEXT2: O presidente do ...,1.0
38989,"TEXT1: Gyde Jensen; TEXT2: Na Alemanha, a pres...",0.0
38990,"TEXT1: Angela Merkel; TEXT2: Na Alemanha, a pr...",0.0


In [118]:
data

,id,name,input,prediction
0,54006424,Clemens Tönnies,TEXT1: Clemens Tönnies; TEXT2: O presidente d...,1.0
1,54006424,Gerald Asamoah,TEXT1: Gerald Asamoah; TEXT2: No mês de agost...,1.0
2,54006424,Hans Sarpei,"TEXT1: Hans Sarpei; TEXT2: No mês de agosto, ...",1.0
3,54006424,Cacau,"TEXT1: Cacau; TEXT2: No mês de agosto, durant...",1.0
4,54003004,Angela Merkel,TEXT1: Angela Merkel; TEXT2: Em meados de marc...,0.0
...,...,...,...,...
38987,54005845,Von der Leyen,TEXT1: Von der Leyen; TEXT2: Von der Leyen acr...,1.0
38988,54005845,Charles Michel,TEXT1: Charles Michel; TEXT2: O presidente do ...,1.0
38989,54005845,Gyde Jensen,"TEXT1: Gyde Jensen; TEXT2: Na Alemanha, a pres...",0.0
38990,54005845,Angela Merkel,"TEXT1: Angela Merkel; TEXT2: Na Alemanha, a pr...",0.0


In [120]:
np.all(data.input == data_with_prediction.input)

True

In [121]:
data.input == data_with_prediction.input

0        True
1        True
2        True
3        True
4        True
         ... 
38987    True
38988    True
38989    True
38990    True
38991    True
Name: input, Length: 38992, dtype: bool

In [125]:
outputdir = Path.cwd()
filename = 'portuguese_all_processed_articles_v2.csv'
data.to_csv(outputdir/filename, index=False)